In [ ]:
import polars as pl

In [ ]:
s3_key_prefix = "s3_key_prefix"

manifest = pl.read_json("./data/2025-05-27T01-00Z/manifest.json")
manifest = manifest["files"][0].to_list()
manifest = [meta["key"].replace(s3_key_prefix, "./data") for meta in manifest]

In [ ]:
import shutil
import os

for file_path in manifest:
    shutil.move(src=file_path, dst=os.path.join("data", "latest", os.path.basename(file_path)))

In [ ]:
len(os.listdir(os.path.join("data", "latest"))) == len(manifest)

In [ ]:
len(manifest)

In [ ]:
df = pl.scan_parquet(manifest)
df

In [ ]:
# stroage_class 별 사이즈 집꼐
df_agg =  df.group_by("storage_class").agg([
    pl.len().alias("count"),
    pl.col("size").sum().alias("total_size"),
    pl.col("size").mean().alias("avg_size"),
    pl.col("size").min().alias("min_size"),
    pl.col("size").max().alias("max_size")
]).sort("total_size", descending=True).collect(streaming=True)

In [ ]:
# bucket 별 사이즈 집계
df_agg = df.group_by("bucket").agg([
    pl.len().alias("count"),
    pl.col("size").sum().alias("total_size"),
    pl.col("size").mean().alias("avg_size"),
    pl.col("size").min().alias("min_size"),
    pl.col("size").max().alias("max_size")
]).sort("count", descending=True).collect(streaming=True)

In [ ]:
# s3 key prefix(3 level) 별 사이즈 집계
df_res = df.with_columns(
    [pl.col("key").str.split_exact("/", n=3).struct.rename_fields(["level1", "level2", "level3"])]
).unnest(
    "key"
).group_by(
    ["level1", "level2", "level3"]
).agg(
    [
        pl.len().alias("count"),
        pl.col("size").sum().alias("total_size"),
        pl.col("size").mean().alias("avg_size")
    ]
).sort("count", descending=True).collect(streaming=True)

In [ ]:
df_res

In [ ]:
df_res.write_csv("data.csv")